<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/mmdetection/Train_an_Object_Detection+Semantic_Segmentation_Model_with_MMDetection_and_W&B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{mmdetection-wandb-colab} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases"/> <br>

<!--- @wandbcode{mmdetection-wandb-colab, v=1} -->

<img src="http://wandb.me/mini-diagram" width="600" alt="Weights & Biases"/>

# 💡 Train an Object Detection+Semantic Segmentation Model with MMDetection and Weights and Biases

Weights and Biases

In this colab, we will train a Mask RCNN model using [MMDetection](https://mmdetection.readthedocs.io/en/latest/1_exist_data_model.html) on a small [Balloon](https://github.com/matterport/Mask_RCNN/tree/master/samples/balloon) dataset.Through this colab you will learn to:

* use MMDetection to train an object detection + Semantic Segmentation model on a custom dataset,
* use [Weights and Biases](https://wandb.ai/site) to log training and validation metrics, visualize model predictions, visualize raw validation dataset, and more.

This colab in particular, will showcase a dedicated [`MMDetWandbHook`](https://github.com/open-mmlab/mmdetection/pull/7459) for MMDetection that can be used to:

✅ Log training and evaluation metrics. <br>
✅ Log versioned model checkpoints. <br>
✅ Log versioned validation dataset with ground truth bounding boxes and segmentation masks. <br>
✅ Log and visualize model predictions.

But before we continue, here's a quick summary of MMDetection and W&B if you are not familiar with them.

### 📸 MMDetection

MMDetection is an open source object detection toolbox based on PyTorch. It provides composable components that are easy to customize and has out-of-box support for single and multi GPU training/inference. It also has hundreds of pretrained detection models in Model Zoo, and supports multiple standard datasets. Check out the GitHub repository [here](https://github.com/open-mmlab/mmdetection).

### 📸 Weights and Biases

Consider **[Weights and Biases](https://wandb.ai/site)** (W&B) to be the GitHub for machine learning. Use W&B for machine learning experiment tracking, dataset and model versioning, project collaboration, hyperparameter optimization, dataset exploration, model evaluation and so much more. If you are new to W&B, check out this [intro colab](https://wandb.me/intro).

# ⚽️ Imports and Setup

### 1️⃣ Install MMDetection

MMDetection is heavily dependent on the [MMCV](https://mmcv.readthedocs.io/en/latest/#installation) library. We will have to install the version of MMCV that is compatible with the given PyTorch version. Check out the [Installation documentation](https://mmdetection.readthedocs.io/en/latest/get_started.html#installation) for more details. 

In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install -qq mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!pip install git+git://github.com/open-mmlab/mmdetection.git

### 2️⃣ Install Weights and Biases

Install the latest version of W&B. 

In [ ]:
!pip install -qU wandb

### 3️⃣ General Imports

In [ ]:
import os
import os.path as osp
import torch
import torchvision
import numpy as np

# MMDetection
import mmdet
print(mmdet.__version__)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import set_random_seed

# MMCV
import mmcv
from mmcv import Config

# Weights and Biases
import wandb
print(wandb.__version__)

### 4️⃣ Login with you W&B account

Create a free W&B account (it's free for personal and academic usage). Visit [wandb.ai/authorize](https://wandb.ai/authorize) to get your unique authentication token. 

In [ ]:
wandb.login()

# 🏀 Dataset

We will be using a small [Balloon](https://github.com/matterport/Mask_RCNN/tree/master/samples/balloon) dataset for this colab notebook.

### 1️⃣ Download the dataset

In [ ]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
!unzip -q balloon_dataset.zip

In [ ]:
!ls balloon

> Note: The `train` and `val` folders contain the images along with annotations as `.json` files.

### 2️⃣ Convert annotations to COCO format

To support a new data format, it's recommended to convert the annotations to COCO format or PASCAL VOC format.

If you are converting annotations to COCO format, do so offline and use the `CocoDataset` class. If you are converting it to the PASCAL format, use the `VOCDataset` class. You will see the usage below.

You can find more details about customizing the dataset [here](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_dataset.html).

In [ ]:
#@title Utility function `convert_balloon_to_coco`
# Credit: https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_InstanceSeg_Tutorial.ipynb
def convert_balloon_to_coco(ann_file, out_file, image_prefix):
    data_infos = mmcv.load(ann_file)

    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(mmcv.track_iter_progress(data_infos.values())):
        filename = v['filename']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(dict(
            id=idx,
            file_name=filename,
            height=height,
            width=width))

        bboxes = []
        labels = []
        masks = []
        for _, obj in v['regions'].items():
            assert not obj['region_attributes']
            obj = obj['shape_attributes']
            px = obj['all_points_x']
            py = obj['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            x_min, y_min, x_max, y_max = (
                min(px), min(py), max(px), max(py))


            data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=0,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=[poly],
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'balloon'}])
    mmcv.dump(coco_format_json, out_file)

In [ ]:
convert_balloon_to_coco(
    'balloon/train/via_region_data.json',
    'balloon/train/annotation_coco.json',
    'balloon/train/')
convert_balloon_to_coco(
    'balloon/val/via_region_data.json',
    'balloon/val/annotation_coco.json',
    'balloon/val/')

# 🏈 Model

There are over hundred pre-trained object detectors provided by MMDetection via Model Zoo. Check out the Model Zoo [documentation](https://mmdetection.readthedocs.io/en/v2.21.0/model_zoo.html) page.

You can also customize the model's backbone, neck, head, ROI, and loss. More on customizing the model [here](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_models.html).

### 1️⃣ Download the model

We will be using a pretrained model checkpoint to fine tune on our custom dataset. Let's download the model in the `checkpoints` directory.

I am using the [this](https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth) model with [this](https://github.com/open-mmlab/mmdetection/tree/master/configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py) config from the Model Zoo. You can find similar Mask RCNN model [here](https://github.com/open-mmlab/mmdetection/tree/master/configs/mask_rcnn).

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

# ⚾️ Configuration

MMDetection relies heavily on a config system. In the cell below, we will be loading a config file and modify few of the methods as per the need of this notebook.

Note that both train and test dataloaders will use the same training samples. This is not a recommended practice but for the sake of a simplified notebook, let's use it. 

Learn more about the MMDetection Config system [here](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html).

### 1️⃣ Load the config file

In [ ]:
config_file = 'mmdetection/configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py'
cfg = Config.fromfile(config_file)

### 2️⃣ Modify data config

In [ ]:
# Define type and path to the images.
cfg.dataset_type = 'COCODataset'

cfg.data.test.ann_file = 'balloon/val/annotation_coco.json'
cfg.data.test.img_prefix = 'balloon/val/'
cfg.data.test.classes = ('balloon',)

cfg.data.train.ann_file = 'balloon/train/annotation_coco.json'
cfg.data.train.img_prefix = 'balloon/train/'
cfg.data.train.classes = ('balloon',)

cfg.data.val.ann_file = 'balloon/val/annotation_coco.json'
cfg.data.val.img_prefix = 'balloon/val/'
cfg.data.val.classes = ('balloon',)

### 3️⃣ Modify model config

In [ ]:
# modify num classes of the model in box head and mask head
cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.mask_head.num_classes = 1

# Use the pretrained model.
cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

### 4️⃣ Modify training config

In [ ]:
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Epochs
cfg.runner.max_epochs = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# ⭐️ Set the checkpoint interval.
cfg.checkpoint_config.interval = 4

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'
cfg.device = 'cuda'

### 5️⃣ Modify evaluation config

In [ ]:
# ⭐️ Set the evaluation interval.
cfg.evaluation.interval = 2

> Note that we are evaluating the model on the validation dataset after every 2 epochs while saving the checkpoints after every 4 epochs.

⭐️ If you want to log the configuration to W&B for efficient experiment tracking, add the config filename to a dict with key `exp_name`.

In [ ]:
meta = dict()
meta['exp_name'] = osp.basename(config_file)
print(meta)

# 🎾 Define Weights and Biases Hook

MMDetection comes with a dedicated Weights and Biases Hook - `MMDetWandHook`.

With this dedicated hook, you can:

* log train and eval metrics along with system (CPU/GPU) metrics, 
* visualize the validation dataset as interactive [W&B Tables](https://docs.wandb.ai/guides/data-vis),
* visualize the model prediction as interactive W&B Tables, and
* save the model checkpoints as [W&B Artifacts](https://docs.wandb.ai/guides/artifacts).

To use this hook, you can append a dict to `log_config.hooks`. The `log_config` wraps multiple logger hooks like  the `TextLoggerHook` used below.

There are four important arguments in the `MMDetWandbHook` that can help you get the most out of MMDetection. 

- `init_kwargs`: Use this argument to in-turn pass arguments to `wandb.init`. You can use it to set the W&B project name, set the team name (entity) if you want to log the runs to a team account, pass the configuration, and more. Check out the arguments that you can pass to `wandb.init` [here](https://docs.wandb.ai/ref/python/init).

- `log_checkpoint`: Save the checkpoint at every checkpoint interval
as W&B Artifacts. Use this for model versioning where each version is a checkpoint. The checkpoint interval is set using `checkpoint_config.interval` (starred above). Note that this feature is dependent on MMCV's [`CheckpointHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.CheckpointHook).

- `log_checkpoint_metadata`: Log the evaluation metrics computed on the validation data with the checkpoint, along with current epoch as a metadata to that checkpoint.

- `num_eval_images`: At every evaluation interval, the `MMDetWandbHook` logs the model prediction as interactive W&B Tables. The eval interval is determined by `evaluation.interval` (starred above). The number of samples logged is given by `num_eval_images`. Before training begins, the callback logs the validation data (images along with bounding box and masks). At every evaluation interval, the model predictions are logged. This Feature is dependent on MMCV's [`EvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.EvalHook) or [`DistEvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.DistEvalHook).

- `bbox_score_thr`: Threshold for bounding box scores.

In [ ]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': 'MMDetection-tutorial'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=10)]

# 🏐 Train

Now that we have the dataset, pretrained model weight, and have defined the configs. Let's stitch them together to train an object detector.

### 1️⃣ Build the Dataset

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

### 2️⃣ Build the Model

In [ ]:
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

### 3️⃣ Train with W&B

In [ ]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# dump config
cfg.dump(osp.join(cfg.work_dir, meta['exp_name']))

# Train
train_detector(model, datasets, cfg, distributed=False, validate=True, meta=meta)

### 4️⃣ Notes on using `MMDetWandbHook`. 

Using `MMDetWandbHook` is easy and in most cases it will throw friendly `UserWarning` if something is not quite right. However in the best interest, here are some of things and best practices you should keep in mind:

* The `MMDetWandbHook` depends on `CheckpointHook` for logging the checkpoints as W&B Artifacts and `EvalHook`/`DistEvalHook` for logging validation data and model predictions. If anyone or both aren't available, this hook will give `UserWarning` and not cause any error. 

* The priority of both `CheckpointHook` and `EvalHook`/`DistEvalHook` should be more than `MMDetWandbHook`. 

* The validation data is logged once as `val_data` W&B Table. The evaluation tables, use reference to this data thus you will not be uploading the same data multiple times. 

* If you want to log the configuration to W&B, pass this key-value pair `'config': cfg._cfg_dict.to_dict()` to `init_kwargs`. However, it's recommended to use `meta['exp_name'] = config_filename` and pass `meta` to `train_detector`.